# SHAP feature selection #
## Code to select feature with combination of LightGBM and SHAP ##

## Parameters cell ##

Parameters are overiddent by papermill when run inside DVC stages



In [1]:
number_of_folds = 5 # this sets global setting of which how many bootstraps to use
repeats = 10
#first round of optimization
lgb_params = {"bagging_fraction": 0.9522534844058304, 
              "boosting_type": "dart", 
              "objective": "regression",
              "feature_fraction": 0.42236910941558053, 
              "lambda_l1": 0.020847266580277746, 
              "lambda_l2": 2.8448564854773326, 
              "learning_rate": 0.11484015430016059, 
              "max_depth": 3, 
              "max_leaves": 35, 
              "min_data_in_leaf": 9}
debug_local = True #to use local version

In [2]:
from pathlib import Path
import sys
import inspect

local = (Path("..") / "yspecies").resolve()
if debug_local and local.exists():
  sys.path.insert(0, Path("..").as_posix())
  #sys.path.insert(0, local.as_posix())
  print("extending pathes with local yspecies")
  print(sys.path)
  %load_ext autoreload
  %autoreload 2

extending pathes with local yspecies
['..', '/data/sources/yspecies/notebooks', '/opt/miniconda3/envs/yspecies/lib/python38.zip', '/opt/miniconda3/envs/yspecies/lib/python3.8', '/opt/miniconda3/envs/yspecies/lib/python3.8/lib-dynload', '', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages', '/opt/miniconda3/envs/yspecies/lib/python3.8/site-packages/IPython/extensions', '/home/antonkulaga/.ipython']


In [3]:
from dataclasses import dataclass, replace
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt

In [4]:
from typing import *
from yspecies.dataset import *
from yspecies.utils import *
from yspecies.workflow import TupleWith, Repeat
from yspecies.config import *
from yspecies.preprocess import FeatureSelection, DataExtractor
from yspecies.partition import DataPartitioner, PartitionParameters
from yspecies.models import Metrics
from yspecies.selection import ShapSelector

In [5]:
#settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
import pprint
pp = pprint.PrettyPrinter(indent=4)

#charts settings
import matplotlib.pyplot as plt
from IPython.display import set_matplotlib_formats
%matplotlib auto
plt.ioff()
set_matplotlib_formats('svg')

Using matplotlib backend: TkAgg


### Loading data ###
Let's load data from species/genes/expressions selected by select_samples.py notebook

In [6]:
from pathlib import Path
locations: Locations = Locations("./") if Path("./data").exists() else Locations("../")

In [7]:
data = ExpressionDataset.from_folder(locations.interim.selected)
data

expressions,genes,species,samples,Genes Metadata,Species Metadata
"(445, 12340)",12340,39,445,"(12340, 2)","(40, 19)"


## Setting up SHAP selection pipeline ##

Deciding on selection parameters (which fields to include, exclude, predict)

In [8]:
partition_params = PartitionParameters(number_of_folds, 0, 2, [],  42)


In [9]:
partition_shap_pipe = Pipeline([
    ("partitioner", DataPartitioner()),
    ('prepare_for_partitioning', TupleWith(lgb_params)),
    ("shap_computation", ShapSelector())
]
)
repeated_cv =  Repeat(partition_shap_pipe, repeats, lambda x,i: (x[0], replace(x[1], seed = i)))

In [10]:
selection_pipeline =  Pipeline([
    ('extractor', DataExtractor()),
    ('prepare_for_partitioning', TupleWith(partition_params)), # to extract the data required for ML from the dataset
    ("partition_shap", repeated_cv)]
    )

## Setting up features to select ##

In [11]:
selection = select_lifespan = FeatureSelection(
    samples = ["tissue","species"], #samples metadata to include
    species =  [], #species metadata other then Y label to include
    exclude_from_training = ["species"],  #exclude some fields from LightGBM training
    to_predict = "lifespan", #column to predict
    categorical = ["tissue"])

In [12]:
select_lifespan = selection
select_mass = replace(selection, to_predict = "mass_g")
select_gestation = replace(selection, to_predict = "gestation")
select_mtgc = replace(selection, to_predict = "mtgc")

# First stage selection #

In [13]:
stage_one_lifespan = selection_pipeline.fit_transform((data, select_lifespan))
stage_one_lifespan

Early stopping is not available in dart mode


[200]	valid_0's l2: 38.7282
[200]	valid_0's l2: 14.7218
[200]	valid_0's l2: 2070.36
[200]	valid_0's l2: 38.694
[200]	valid_0's l2: 123.122


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Early stopping is not available in dart mode


[200]	valid_0's l2: 133.245
[200]	valid_0's l2: 24.8638
[200]	valid_0's l2: 43.6909
[200]	valid_0's l2: 17.2242
[200]	valid_0's l2: 30.18


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Early stopping is not available in dart mode


...

[200]	valid_0's l2: 22.1786
[200]	valid_0's l2: 59.7035
[200]	valid_0's l2: 38.8822
[200]	valid_0's l2: 79.7684
[200]	valid_0's l2: 156.484


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Early stopping is not available in dart mode


[200]	valid_0's l2: 34.2988
[200]	valid_0's l2: 2058.34
[200]	valid_0's l2: 102.973
[200]	valid_0's l2: 27.8676
[200]	valid_0's l2: 22.3528


Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.
Setting feature_perturbation = "tree_path_dependent" because no background data was given.


[<table border='2'><caption><h3>Feature selection results</h3><caption><tr style='text-align:center'><th>selected</th><th>metrics</th></tr><tr><td><div>
 <style scoped>
     .dataframe tbody tr th:only-of-type {
         vertical-align: middle;
     }
 
     .dataframe tbody tr th {
         vertical-align: top;
     }
 
     .dataframe thead th {
         text-align: right;
     }
 </style>
 <table border="1" class="dataframe">
   <thead>
     <tr style="text-align: right;">
       <th></th>
       <th>symbol</th>
       <th>gain_score_to_lifespan</th>
       <th>kendall_tau_to_lifespan</th>
     </tr>
     <tr>
       <th>ensembl_id</th>
       <th></th>
       <th></th>
       <th></th>
     </tr>
   </thead>
   <tbody>
     <tr>
       <th>ENSG00000167515</th>
       <td>TRAPPC2L</td>
       <td>124386.632</td>
       <td>-0.237</td>
     </tr>
     <tr>
       <th>ENSG00000060762</th>
       <td>MPC1</td>
       <td>115967.760</td>
       <td>-0.443</td>
     </tr>
     <tr>
       <th>ENSG00000066923</th>
       <td>STAG3</td>
       <td>42371.166</td>
       <td>0.490</td>
     </tr>
     <tr>
       <th>ENSG00000136436</th>
       <td>CALCOCO2</td>
       <td>30505.962</td>
       <td>0.762</td>
     </tr>
     <tr>
       <th>ENSG00000105672</th>
       <td>ETV2</td>
       <td>30083.124</td>
       <td>0.711</td>
     </tr>
     <tr>
       <th>ENSG00000188763</th>
       <td>FZD9</td>
       <td>25725.001</td>
       <td>-0.209</td>
     </tr>
     <tr>
       <th>ENSG00000119616</th>
       <td>FCF1</td>
       <td>25614.721</td>
       <td>-0.225</td>
     </tr>
     <tr>
       <th>ENSG00000188747</th>
       <td>NOXA1</td>
       <td>20298.523</td>
       <td>0.740</td>
     </tr>
     <tr>
       <th>ENSG00000129988</th>
       <td>LBP</td>
       <td>18346.884</td>
       <td>-0.567</td>
     </tr>
     <tr>
       <th>ENSG00000170835</th>
       <td>CEL</td>
       <td>15777.882</td>
       <td>0.612</td>
     </tr>
     <tr>
       <th>ENSG00000142002</th>
       <td>DPP9</td>
       <td>9206.453</td>
       <td>-0.698</td>
     </tr>
     <tr>
       <th>ENSG00000171121</th>
       <td>KCNMB3</td>
       <td>9033.897</td>
       <td>0.638</td>
     </tr>
     <tr>
       <th>ENSG00000176894</th>
       <td>PXMP2</td>
       <td>8290.900</td>
       <td>-0.197</td>
     </tr>
     <tr>
       <th>ENSG00000173567</th>
       <td>ADGRF3</td>
       <td>6378.231</td>
       <td>0.716</td>
     </tr>
     <tr>
       <th>ENSG00000089053</th>
       <td>ANAPC5</td>
       <td>6031.108</td>
       <td>-0.610</td>
     </tr>
     <tr>
       <th>ENSG00000165501</th>
       <td>LRR1</td>
       <td>4597.261</td>
       <td>0.785</td>
     </tr>
     <tr>
       <th>ENSG00000143742</th>
       <td>SRP9</td>
       <td>4558.729</td>
       <td>0.693</td>
     </tr>
     <tr>
       <th>ENSG00000144451</th>
       <td>SPAG16</td>
       <td>4506.909</td>
       <td>0.560</td>
     </tr>
     <tr>
       <th>ENSG00000122034</th>
       <td>GTF3A</td>
       <td>2976.676</td>
       <td>0.505</td>
     </tr>
     <tr>
       <th>ENSG00000143994</th>
       <td>ABHD1</td>
       <td>2366.458</td>
       <td>-0.783</td>
     </tr>
     <tr>
       <th>ENSG00000169583</th>
       <td>CLIC3</td>
       <td>2365.941</td>
       <td>-0.599</td>
     </tr>
     <tr>
       <th>ENSG00000128739</th>
       <td>SNRPN</td>
       <td>2145.065</td>
       <td>-0.533</td>
     </tr>
     <tr>
       <th>ENSG00000144713</th>
       <td>RPL32</td>
       <td>1943.928</td>
       <td>-0.096</td>
     </tr>
     <tr>
       <th>ENSG00000165689</th>
       <td>ENTR1</td>
       <td>1905.874</td>
       <td>-0.721</td>
     </tr>
     <tr>
       <th>ENSG00000101190</th>
       <td>TCFL5</td>
       <td>1383.824</td>
       <td>0.731</td>
     </tr>
     <tr>
       <th>ENSG00000088992</th>
       <td>TESC</td>
       <td>655.047</td>
       <td>0.492</td>
     </tr>
     <tr>
       <th>ENSG00000198663</th>
       <td>C6orf89</td>
       

In [75]:
from yspecies.results import FeatureSummary
summary = FeatureSummary(stage_one_lifespan)
stage_one_lifespan[0]

<table border='2'><caption><h3>Feature selection results</h3><caption><tr style='text-align:center'><th>selected</th><th>metrics</th></tr><tr><td><div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>symbol</th>
      <th>gain_score_to_lifespan</th>
      <th>kendall_tau_to_lifespan</th>
    </tr>
    <tr>
      <th>ensembl_id</th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>ENSG00000276293</th>
      <td>PIP4K2B</td>
      <td>478450.068</td>
      <td>0.048</td>
    </tr>
    <tr>
      <th>ENSG00000083896</th>
      <td>YTHDC1</td>
      <td>263331.356</td>
      <td>0.079</td>
    </tr>
    <tr>
      <th>ENSG00000010219</th>
      <td>DYRK4</td>
      <td>188286.492</td>
      <td>0.246</td>
    </tr>
    <tr>
      <th>ENSG00000167515</th>
      <td>TRAPPC2L</td>
      <td>126939.943</td>
      <td>-0.233</td>
    </tr>
    <tr>
      <th>ENSG00000060762</th>
      <td>MPC1</td>
      <td>121102.080</td>
      <td>-0.438</td>
    </tr>
    <tr>
      <th>ENSG00000204590</th>
      <td>GNL1</td>
      <td>111555.248</td>
      <td>-0.032</td>
    </tr>
    <tr>
      <th>ENSG00000066923</th>
      <td>STAG3</td>
      <td>91786.359</td>
      <td>0.489</td>
    </tr>
    <tr>
      <th>ENSG00000204498</th>
      <td>NFKBIL1</td>
      <td>91750.480</td>
      <td>-0.311</td>
    </tr>
    <tr>
      <th>ENSG00000148248</th>
      <td>SURF4</td>
      <td>84496.348</td>
      <td>-0.037</td>
    </tr>
    <tr>
      <th>ENSG00000136436</th>
      <td>CALCOCO2</td>
      <td>29367.246</td>
      <td>0.757</td>
    </tr>
    <tr>
      <th>ENSG00000105672</th>
      <td>ETV2</td>
      <td>28524.260</td>
      <td>0.698</td>
    </tr>
    <tr>
      <th>ENSG00000188747</th>
      <td>NOXA1</td>
      <td>21349.208</td>
      <td>0.705</td>
    </tr>
    <tr>
      <th>ENSG00000164879</th>
      <td>CA3</td>
      <td>20272.084</td>
      <td>-0.505</td>
    </tr>
    <tr>
      <th>ENSG00000184983</th>
      <td>NDUFA6</td>
      <td>16096.294</td>
      <td>-0.563</td>
    </tr>
    <tr>
      <th>ENSG00000129988</th>
      <td>LBP</td>
      <td>14307.624</td>
      <td>-0.563</td>
    </tr>
    <tr>
      <th>ENSG00000170835</th>
      <td>CEL</td>
      <td>10981.792</td>
      <td>0.675</td>
    </tr>
    <tr>
      <th>ENSG00000157343</th>
      <td>ARMC12</td>
      <td>9844.675</td>
      <td>0.692</td>
    </tr>
    <tr>
      <th>ENSG00000065268</th>
      <td>WDR18</td>
      <td>9520.684</td>
      <td>-0.789</td>
    </tr>
    <tr>
      <th>ENSG00000142002</th>
      <td>DPP9</td>
      <td>9508.972</td>
      <td>-0.712</td>
    </tr>
    <tr>
      <th>ENSG00000184985</th>
      <td>SORCS2</td>
      <td>8635.959</td>
      <td>-0.401</td>
    </tr>
    <tr>
      <th>ENSG00000171121</th>
      <td>KCNMB3</td>
      <td>8605.678</td>
      <td>0.638</td>
    </tr>
    <tr>
      <th>ENSG00000129187</th>
      <td>DCTD</td>
      <td>8143.963</td>
      <td>0.683</td>
    </tr>
    <tr>
      <th>ENSG00000122952</th>
      <td>ZWINT</td>
      <td>7606.769</td>
      <td>-0.653</td>
    </tr>
    <tr>
      <th>ENSG00000144451</th>
      <td>SPAG16</td>
      <td>7404.543</td>
      <td>0.544</td>
    </tr>
    <tr>
      <th>ENSG00000089053</th>
      <td>ANAPC5</td>
      <td>5370.228</td>
      <td>-0.606</td>
    </tr>
    <tr>
      <th>ENSG00000105993</th>
      <td>DNAJB6</td>
      <td>4856.146</td>
      <td>0.329</td>
    </tr>
    <tr>
      <th>ENSG00000152705</th>
      <td>CATSPER3</td>
      <td>3940.447</td>
      <td>0.601</td>
    </tr>
    <tr>
      <th>ENSG00000135845</th>
      <td>PIGC</td>
      <td>2892.276</td>
      <td>0.350</td>
    </tr>
    <

In [15]:
summary.selected.sort_values("kendall_tau_0", ascending=False)["symbol"].values

array(['CALCOCO2', 'MPG', 'NOXA1', 'ETV2', 'CEL', 'ARMC12', 'HRH4',
       'DCTD', 'KCNMB3', 'ADGRF3', 'TIMP1', 'TSPAN10', 'STAG3', 'GUCY2C',
       'DNAJB6', 'LIN7B', 'TRAPPC2L', 'MPC1', 'CTBS', 'CA3', 'NDUFA6',
       'LBP', 'PUSL1', 'MPC2', 'SNRPN', 'ADPRM', 'DPP9', 'WDR18'],
      dtype=object)

In [73]:
"gain" if len([c for c in summary.results[0].selected.columns if "gain" in c])>0 else "shap"

'gain'